# Построение модели регрессии: подготовка и визуализация данных

## **Линейная регрессия для тыкв - Урок 2**
#### Введение

Теперь, когда у вас есть все необходимые инструменты для начала работы с построением моделей машинного обучения с использованием Tidymodels и Tidyverse, вы готовы начать задавать вопросы вашим данным. Работая с данными и применяя решения на основе машинного обучения, крайне важно уметь задавать правильные вопросы, чтобы максимально раскрыть потенциал вашего набора данных.

В этом уроке вы узнаете:

- Как подготовить данные для построения модели.

- Как использовать `ggplot2` для визуализации данных.

Вопрос, на который вы хотите получить ответ, определит, какой тип алгоритмов машинного обучения вы будете использовать. А качество полученного ответа будет во многом зависеть от характера ваших данных.

Давайте разберем это на практическом примере.

<p >
   <img src="../../images/unruly_data.jpg"
   width="700"/>
   <figcaption>Иллюстрация от @allison_horst</figcaption>


## 1. Импорт данных о тыквах и вызов Tidyverse

Для работы с данными в этом уроке нам понадобятся следующие пакеты:

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/) — это [набор пакетов для R](https://www.tidyverse.org/packages), созданный для того, чтобы сделать работу с данными быстрее, проще и интереснее!

Вы можете установить их следующим образом:

`install.packages(c("tidyverse"))`

Скрипт ниже проверяет, установлены ли у вас необходимые пакеты для выполнения этого модуля, и устанавливает их, если каких-то не хватает.


In [ ]:
suppressWarnings(if(!require("pacman")) install.packages("pacman"))
pacman::p_load(tidyverse)

Теперь давайте запустим несколько пакетов и загрузим [данные](https://github.com/microsoft/ML-For-Beginners/blob/main/2-Regression/data/US-pumpkins.csv), предоставленные для этого урока!


In [ ]:
# Load the core Tidyverse packages
library(tidyverse)

# Import the pumpkins data
pumpkins <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/2-Regression/data/US-pumpkins.csv")


# Get a glimpse and dimensions of the data
glimpse(pumpkins)


# Print the first 50 rows of the data set
pumpkins %>% 
  slice_head(n =50)

Быстрый вызов `glimpse()` сразу показывает, что есть пропуски и смесь строковых данных (`chr`) и числовых данных (`dbl`). Поле `Date` имеет тип "символы", а также есть странный столбец под названием `Package`, где данные представляют собой смесь значений, таких как `sacks`, `bins` и других. Данные, по сути, немного хаотичны 😤.

На самом деле, довольно редко встречается набор данных, который полностью готов к использованию для создания модели машинного обучения прямо "из коробки". Но не переживайте, в этом уроке вы научитесь подготавливать необработанный набор данных с использованием стандартных библиотек R 🧑‍🔧. Вы также освоите различные техники визуализации данных. 📈📊
<br>

> Напоминание: Оператор pipe (`%>%`) выполняет операции в логической последовательности, передавая объект дальше в функцию или выражение. Вы можете представить оператор pipe как выражение "а затем" в вашем коде.


## 2. Проверка на отсутствие данных

Одна из самых распространенных проблем, с которыми сталкиваются специалисты по данным, — это неполные или отсутствующие данные. В R отсутствующие или неизвестные значения обозначаются специальным значением `NA` (Not Available).

Как же определить, что в датафрейме есть пропущенные значения?
<br>
-   Один из простых способов — использовать базовую функцию R `anyNA`, которая возвращает логические значения `TRUE` или `FALSE`.


In [ ]:
pumpkins %>% 
  anyNA()

Отлично, похоже, что некоторые данные отсутствуют! Это хорошее место для начала.

-   Другой способ — использовать функцию `is.na()`, которая показывает, какие элементы столбца отсутствуют, возвращая логическое значение `TRUE`.


In [ ]:
pumpkins %>% 
  is.na() %>% 
  head(n = 7)

Хорошо, задача выполнена, но с таким большим датафреймом это было бы неэффективно и практически невозможно просмотреть все строки и столбцы по отдельности😴.

-   Более интуитивный способ — это вычислить сумму пропущенных значений для каждого столбца:


In [ ]:
pumpkins %>% 
  is.na() %>% 
  colSums()

Гораздо лучше! Данных не хватает, но, возможно, это не будет иметь значения для выполнения задачи. Посмотрим, что покажет дальнейший анализ.

> Вместе с потрясающим набором пакетов и функций, R обладает очень хорошей документацией. Например, используйте `help(colSums)` или `?colSums`, чтобы узнать больше о функции.


## 3. Dplyr: Грамматика манипуляций с данными

<p >
   <img src="../../images/dplyr_wrangling.png"
   width="569"/>
   <figcaption>Иллюстрация от @allison_horst</figcaption>


<!--![Иллюстрация от \@allison_horst](../../../../../../translated_images/dplyr_wrangling.f5f99c64fd4580f1377fee3ea428b6f8fd073845ec0f8409d483cfe148f0984e.ru.png)<br/>Иллюстрация от \@allison_horst-->


[`dplyr`](https://dplyr.tidyverse.org/), пакет из Tidyverse, представляет собой грамматику манипуляции данными, которая предлагает набор удобных глаголов для решения наиболее распространённых задач обработки данных. В этом разделе мы рассмотрим некоторые из глаголов dplyr!  
<br>


#### dplyr::select()

`select()` — это функция из пакета `dplyr`, которая помогает выбрать столбцы для сохранения или исключения.

Чтобы упростить работу с вашим датафреймом, удалите несколько его столбцов, используя `select()`, оставив только те, которые вам нужны.

Например, в этом упражнении наш анализ будет включать столбцы `Package`, `Low Price`, `High Price` и `Date`. Давайте выберем эти столбцы.


In [ ]:
# Select desired columns
pumpkins <- pumpkins %>% 
  select(Package, `Low Price`, `High Price`, Date)


# Print data set
pumpkins %>% 
  slice_head(n = 5)

#### dplyr::mutate()

`mutate()` — это функция из пакета `dplyr`, которая помогает создавать или изменять столбцы, сохраняя существующие столбцы.

Общая структура функции `mutate` выглядит так:

`data %>%   mutate(new_column_name = what_it_contains)`

Давайте попробуем использовать `mutate` на примере столбца `Date`, выполняя следующие операции:

1. Преобразуем даты (в настоящее время они имеют тип character) в формат месяца (это даты США, поэтому формат — `MM/DD/YYYY`).

2. Извлечем месяц из дат в новый столбец.

В R пакет [lubridate](https://lubridate.tidyverse.org/) упрощает работу с данными типа Date-time. Поэтому давайте использовать `dplyr::mutate()`, `lubridate::mdy()`, `lubridate::month()` и посмотрим, как достичь указанных целей. Мы можем удалить столбец Date, так как он больше не понадобится в последующих операциях.


In [ ]:
# Load lubridate
library(lubridate)

pumpkins <- pumpkins %>% 
  # Convert the Date column to a date object
  mutate(Date = mdy(Date)) %>% 
  # Extract month from Date
  mutate(Month = month(Date)) %>% 
  # Drop Date column
  select(-Date)

# View the first few rows
pumpkins %>% 
  slice_head(n = 7)

Ура! 🤩

Теперь давайте создадим новый столбец `Price`, который будет представлять среднюю цену тыквы. Для этого возьмем среднее значение из столбцов `Low Price` и `High Price`, чтобы заполнить новый столбец Price.
<br>


In [ ]:
# Create a new column Price
pumpkins <- pumpkins %>% 
  mutate(Price = (`Low Price` + `High Price`)/2)

# View the first few rows of the data
pumpkins %>% 
  slice_head(n = 5)

Дааа!💪

"Но постойте!", скажете вы, пробежавшись по всему набору данных с помощью `View(pumpkins)`, "Здесь что-то странное!"🤔

Если вы посмотрите на столбец `Package`, то увидите, что тыквы продаются в самых разных конфигурациях. Некоторые продаются в мерах `1 1/9 bushel`, некоторые в мерах `1/2 bushel`, некоторые поштучно, некоторые на вес (в фунтах), а некоторые в больших коробках с разной шириной.

Давайте это проверим:


In [ ]:
# Verify the distinct observations in Package column
pumpkins %>% 
  distinct(Package)

Удивительно!👏

Тыквы, похоже, очень сложно взвешивать с одинаковой точностью, поэтому давайте отфильтруем их, выбрав только те тыквы, в которых строка *bushel* находится в столбце `Package`, и поместим это в новый датафрейм `new_pumpkins`.


#### dplyr::filter() и stringr::str_detect()

[`dplyr::filter()`](https://dplyr.tidyverse.org/reference/filter.html): создает подмножество данных, содержащее только **строки**, которые удовлетворяют вашим условиям, в данном случае тыквы со строкой *bushel* в столбце `Package`.

[stringr::str_detect()](https://stringr.tidyverse.org/reference/str_detect.html): определяет наличие или отсутствие шаблона в строке.

Пакет [`stringr`](https://github.com/tidyverse/stringr) предоставляет простые функции для выполнения распространенных операций со строками.


In [ ]:
# Retain only pumpkins with "bushel"
new_pumpkins <- pumpkins %>% 
       filter(str_detect(Package, "bushel"))

# Get the dimensions of the new data
dim(new_pumpkins)

# View a few rows of the new data
new_pumpkins %>% 
  slice_head(n = 5)

Вы можете видеть, что мы сократили данные до примерно 415 строк, содержащих тыквы в бушелях.🤩
<br>


#### dplyr::case_when()

**Но подождите! Есть еще кое-что, что нужно сделать**

Вы заметили, что количество бушелей варьируется от строки к строке? Вам нужно нормализовать цены, чтобы показывать стоимость за бушель, а не за 1 1/9 или 1/2 бушеля. Пора заняться математикой, чтобы стандартизировать это.

Мы будем использовать функцию [`case_when()`](https://dplyr.tidyverse.org/reference/case_when.html), чтобы *изменить* столбец Price в зависимости от некоторых условий. `case_when` позволяет векторизовать несколько выражений `if_else()`.


In [ ]:
# Convert the price if the Package contains fractional bushel values
new_pumpkins <- new_pumpkins %>% 
  mutate(Price = case_when(
    str_detect(Package, "1 1/9") ~ Price/(1 + 1/9),
    str_detect(Package, "1/2") ~ Price/(1/2),
    TRUE ~ Price))

# View the first few rows of the data
new_pumpkins %>% 
  slice_head(n = 30)

Теперь мы можем проанализировать стоимость за единицу, основываясь на их измерении в бушелях. Однако все это изучение бушелей тыкв показывает, насколько `важно` `понимать природу ваших данных`!

> ✅ Согласно [The Spruce Eats](https://www.thespruceeats.com/how-much-is-a-bushel-1389308), вес бушеля зависит от типа продукта, так как это объемная мера. "Бушель помидоров, например, должен весить 56 фунтов... Листья и зелень занимают больше места при меньшем весе, поэтому бушель шпината весит всего 20 фунтов." Все это довольно сложно! Давайте не будем заморачиваться с переводом бушелей в фунты, а вместо этого оценим стоимость по бушелям. Однако все это изучение бушелей тыкв показывает, насколько важно понимать природу ваших данных!
>
> ✅ Вы заметили, что тыквы, продаваемые половинными бушелями, очень дорогие? Можете понять, почему? Подсказка: маленькие тыквы намного дороже больших, вероятно, потому что их гораздо больше в одном бушеле, учитывая неиспользованное пространство, занимаемое одной большой полой тыквой для пирога.


Теперь, наконец, ради чистого приключения 💁‍♀️, давайте также переместим столбец "Месяц" на первую позицию, то есть `перед` столбцом "Пакет".

Для изменения позиций столбцов используется `dplyr::relocate()`.


In [ ]:
# Create a new data frame new_pumpkins
new_pumpkins <- new_pumpkins %>% 
  relocate(Month, .before = Package)

new_pumpkins %>% 
  slice_head(n = 7)

Отличная работа!👌 Теперь у вас есть чистый, аккуратный набор данных, на основе которого можно построить новую регрессионную модель!


## 4. Визуализация данных с помощью ggplot2

<p >
   <img src="../../images/data-visualization.png"
   width="600"/>
   <figcaption>Инфографика от Дасани Мадипалли</figcaption>


<!--![Инфографика от Дасани Мадипалли](../../../../../../translated_images/data-visualization.54e56dded7c1a804d00d027543f2881cb32da73aeadda2d4a4f10f3497526114.ru.png){width="600"}-->

Существует *мудрое* высказывание, которое звучит так:

> "Простой график принес больше информации в сознание аналитика данных, чем любое другое устройство." --- Джон Тьюки

Часть работы специалиста по данным заключается в том, чтобы продемонстрировать качество и характер данных, с которыми он работает. Для этого они часто создают интересные визуализации, такие как графики, диаграммы и чарты, показывающие различные аспекты данных. Таким образом, они могут визуально показать взаимосвязи и пробелы, которые иначе было бы сложно обнаружить.

Визуализации также могут помочь определить, какой метод машинного обучения наиболее подходит для данных. Например, диаграмма рассеяния, которая, кажется, следует линии, указывает на то, что данные хорошо подходят для задачи линейной регрессии.

R предлагает несколько систем для создания графиков, но [`ggplot2`](https://ggplot2.tidyverse.org/index.html) является одной из самых элегантных и универсальных. `ggplot2` позволяет создавать графики, **комбинируя независимые компоненты**.

Давайте начнем с простого графика рассеяния для столбцов Price и Month.

В этом случае мы начнем с [`ggplot()`](https://ggplot2.tidyverse.org/reference/ggplot.html), укажем набор данных и эстетическое отображение (с помощью [`aes()`](https://ggplot2.tidyverse.org/reference/aes.html)), а затем добавим слои (например, [`geom_point()`](https://ggplot2.tidyverse.org/reference/geom_point.html)) для создания графика рассеяния.


In [ ]:
# Set a theme for the plots
theme_set(theme_light())

# Create a scatter plot
p <- ggplot(data = new_pumpkins, aes(x = Price, y = Month))
p + geom_point()

Является ли этот график полезным 🤷? Есть ли в нем что-то удивительное?

Он не особенно полезен, так как просто отображает ваши данные в виде набора точек за определенный месяц.
<br>


### **Как сделать это полезным?**

Чтобы графики отображали полезные данные, обычно необходимо как-то сгруппировать данные. Например, в нашем случае нахождение средней цены тыкв за каждый месяц даст больше понимания скрытых закономерностей в наших данных. Это подводит нас к еще одному быстрому обзору **dplyr**:

#### `dplyr::group_by() %>% summarize()`

Групповая агрегация в R может быть легко выполнена с помощью

`dplyr::group_by() %>% summarize()`

-   `dplyr::group_by()` изменяет единицу анализа с полного набора данных на отдельные группы, такие как по месяцам.

-   `dplyr::summarize()` создает новый датафрейм с одним столбцом для каждой переменной группировки и одним столбцом для каждой из указанных вами статистик сводки.

Например, мы можем использовать `dplyr::group_by() %>% summarize()` для группировки тыкв по столбцу **Month**, а затем найти **среднюю цену** для каждого месяца.


In [ ]:
# Find the average price of pumpkins per month
new_pumpkins %>%
  group_by(Month) %>% 
  summarise(mean_price = mean(Price))

Кратко!✨

Категориальные признаки, такие как месяцы, лучше всего представлять с помощью столбчатой диаграммы 📊. Слои, отвечающие за создание столбчатых диаграмм, — это `geom_bar()` и `geom_col()`. Обратитесь к `?geom_bar`, чтобы узнать больше.

Давайте создадим одну!


In [ ]:
# Find the average price of pumpkins per month then plot a bar chart
new_pumpkins %>%
  group_by(Month) %>% 
  summarise(mean_price = mean(Price)) %>% 
  ggplot(aes(x = Month, y = mean_price)) +
  geom_col(fill = "midnightblue", alpha = 0.7) +
  ylab("Pumpkin Price")

🤩🤩 Это более полезная визуализация данных! Кажется, она показывает, что самые высокие цены на тыквы наблюдаются в сентябре и октябре. Соответствует ли это вашим ожиданиям? Почему да или почему нет?

Поздравляем с завершением второго урока 👏! Вы подготовили свои данные для построения модели, а затем раскрыли больше инсайтов с помощью визуализаций!



---

**Отказ от ответственности**:  
Этот документ был переведен с помощью сервиса автоматического перевода [Co-op Translator](https://github.com/Azure/co-op-translator). Хотя мы стремимся к точности, пожалуйста, учитывайте, что автоматические переводы могут содержать ошибки или неточности. Оригинальный документ на его исходном языке следует считать авторитетным источником. Для получения критически важной информации рекомендуется профессиональный перевод человеком. Мы не несем ответственности за любые недоразумения или неправильные интерпретации, возникшие в результате использования данного перевода.
